In [1]:
# https://scikit-learn.org/stable/modules/multiclass.html

In [1]:
# Import libraries
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', 400)
import requests
import matplotlib.pyplot as plt
import json
import re
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [2]:
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [3]:

salaries_df = pd.read_csv('salaries_cleaned.csv')

In [4]:
salaries_df.head()

,Unnamed: 0,timestamp,date,company,level,title,totalyearlycompensation,location,latitude,longitude,yearsofexperience,yearsatcompany,basesalary,stockgrantvalue,bonus,gender,Race,Education
0,0,2017-06-07 11:33:00,2017-06-07 00:00:00,Oracle,L3,Product Manager,127000,"Redwood City, CA",37.486324,-122.232523,1.5,1.5,107000,20000,10000,Unknown,Unknown,Unknown
1,1,2017-06-10 17:11:00,2017-06-10 00:00:00,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",37.779026,-122.419906,5.0,3.0,0,0,0,Unknown,Unknown,Unknown
2,2,2017-06-11 14:53:00,2017-06-11 00:00:00,Amazon,L7,Product Manager,310000,"Seattle, WA",47.603832,-122.330062,8.0,0.0,155000,0,0,Unknown,Unknown,Unknown
3,3,2017-06-17 00:23:00,2017-06-17 00:00:00,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",37.368830,-122.036349,7.0,5.0,157000,180000,35000,Unknown,Unknown,Unknown
4,4,2017-06-20 10:58:00,2017-06-20 00:00:00,Microsoft,60,Software Engineer,157000,"Mountain View, CA",37.389389,-122.083210,5.0,3.0,0,0,0,Unknown,Unknown,Unknown


In [5]:
salaries_df.columns

Index(['Unnamed: 0', 'timestamp', 'date', 'company', 'level', 'title',
       'totalyearlycompensation', 'location', 'latitude', 'longitude',
       'yearsofexperience', 'yearsatcompany', 'basesalary', 'stockgrantvalue',
       'bonus', 'gender', 'Race', 'Education'],
      dtype='object')

In [6]:
salaries_df['company'].value_counts()

Amazon             8208
Microsoft          5252
Google             4367
Facebook           3004
Apple              2055
                   ... 
CSV Health            1
Mckinsey              1
Tune                  1
Jp morgan chase       1
Csg                   1
Name: company, Length: 1076, dtype: int64

In [7]:
val=salaries_df['company'].value_counts()
print(val[val>25])

Amazon           8208
Microsoft        5252
Google           4367
Facebook         3004
Apple            2055
                 ... 
Glassdoor          26
Sony               26
HBO                26
ZS Associates      26
Huawei             26
Name: company, Length: 264, dtype: int64


In [8]:
#companies over 25 jobs
comapanies_to_replace=[]
company=salaries_df['company'].value_counts()
company=pd.DataFrame(company)
company.head(20)

,company
Amazon,8208
Microsoft,5252
Google,4367
Facebook,3004
Apple,2055
Oracle,1143
SalesForce,1065
Intel,991
Cisco,943
IBM,927


In [10]:
comapanies_to_replace.append(company[company.company<15].index)

In [11]:
#Replaced with other for all companies besides the top 20
for x in comapanies_to_replace:
    salaries_df['company'] = salaries_df['company'].replace(x,"Other")
    
# Check to make sure binning was successful
salaries_df['company'].value_counts()

Amazon                  8208
Microsoft               5252
Other                   4895
Google                  4367
Facebook                3004
                        ... 
Adyen                     15
WePay                     15
Kroger                    15
National Instruments      15
WarnerMedia               15
Name: company, Length: 398, dtype: int64

In [45]:
top_company_list_df=pd.DataFrame(salaries_df['company'].value_counts())

In [46]:
top_company_list_df.reset_index(level=0, inplace=True)
top_company_list_df

,index,company
0,Amazon,8208
1,Microsoft,5252
2,Other,4895
3,Google,4367
4,Facebook,3004
...,...,...
393,Adyen,15
394,WePay,15
395,Kroger,15
396,National Instruments,15


In [50]:
top_company_list_df_2=top_company_list_df.head(21)

In [59]:
top_company_list_df_2=top_company_list_df_2.sort_values('index')
top_company_list_df_2

,index,company
0,Amazon,8208
5,Apple,2055
17,Bloomberg,547
12,Capital One,786
9,Cisco,943
4,Facebook,3004
18,Goldman Sachs,474
3,Google,4367
10,IBM,927
8,Intel,991


In [61]:
top_company_list_2=[]
#<option value="Business Analyst">Business Analyst</option>
for i in top_company_list_df_2['index']:
    print(f'<option value="{i}">'+ i + '</option>')

    

<option value="Amazon">Amazon</option>
<option value="Apple">Apple</option>
<option value="Bloomberg">Bloomberg</option>
<option value="Capital One">Capital One</option>
<option value="Cisco">Cisco</option>
<option value="Facebook">Facebook</option>
<option value="Goldman Sachs">Goldman Sachs</option>
<option value="Google">Google</option>
<option value="IBM">IBM</option>
<option value="Intel">Intel</option>
<option value="Intuit">Intuit</option>
<option value="JPMorgan Chase">JPMorgan Chase</option>
<option value="LinkedIn">LinkedIn</option>
<option value="Microsoft">Microsoft</option>
<option value="Oracle">Oracle</option>
<option value="Other">Other</option>
<option value="PayPal">PayPal</option>
<option value="Qualcomm">Qualcomm</option>
<option value="SalesForce">SalesForce</option>
<option value="Uber">Uber</option>
<option value="VMware">VMware</option>


In [12]:
year_df=salaries_df['date'].str.split('-', expand=True)

In [13]:
salaries_df['date']=year_df[0]

In [14]:
salaries_df['month']=year_df[1]

In [15]:
salaries_df

,Unnamed: 0,timestamp,date,company,level,title,totalyearlycompensation,location,latitude,longitude,yearsofexperience,yearsatcompany,basesalary,stockgrantvalue,bonus,gender,Race,Education,month
0,0,2017-06-07 11:33:00,2017,Oracle,L3,Product Manager,127000,"Redwood City, CA",37.486324,-122.232523,1.5,1.5,107000,20000,10000,Unknown,Unknown,Unknown,06
1,1,2017-06-10 17:11:00,2017,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",37.779026,-122.419906,5.0,3.0,0,0,0,Unknown,Unknown,Unknown,06
2,2,2017-06-11 14:53:00,2017,Amazon,L7,Product Manager,310000,"Seattle, WA",47.603832,-122.330062,8.0,0.0,155000,0,0,Unknown,Unknown,Unknown,06
3,3,2017-06-17 00:23:00,2017,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",37.368830,-122.036349,7.0,5.0,157000,180000,35000,Unknown,Unknown,Unknown,06
4,4,2017-06-20 10:58:00,2017,Microsoft,60,Software Engineer,157000,"Mountain View, CA",37.389389,-122.083210,5.0,3.0,0,0,0,Unknown,Unknown,Unknown,06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62637,62637,2018-09-09 11:52:00,2018,Google,T4,Software Engineer,327000,"Seattle, WA",47.603832,-122.330062,10.0,1.0,155000,150000,22000,Unknown,Unknown,Unknown,09
62638,62638,2018-09-13 08:23:00,2018,Microsoft,62,Software Engineer,237000,"Redmond, WA",47.669414,-122.123877,2.0,2.0,146900,73200,16000,Unknown,Unknown,Unknown,09
62639,62639,2018-09-13 14:35:00,2018,Other,63,Software Engineer,220000,"Seattle, WA",47.603832,-122.330062,14.0,12.0,157000,25000,20000,Unknown,Unknown,Unknown,09
62640,62640,2018-09-16 16:10:00,2018,SalesForce,Lead MTS,Software Engineer,280000,"San Francisco, CA",37.779026,-122.419906,8.0,4.0,194688,57000,29000,Unknown,Unknown,Unknown,09


In [16]:
salaries_df['level'].value_counts()

L4                              5014
L5                              4871
L3                              3337
L6                              2871
Senior Software Engineer        1443
                                ... 
Gr 7                               1
Senior BA                          1
Consulting Analyst                 1
Senior Technical Director II       1
Bioinformatics Scientist II        1
Name: level, Length: 2916, dtype: int64

In [17]:
#code to replace levels with fewest value _counts
levels_to_replace=[]
levels=salaries_df['level'].value_counts()
levels=pd.DataFrame(levels)
levels.head(20)

,level
L4,5014
L5,4871
L3,3337
L6,2871
Senior Software Engineer,1443
L2,1163
Senior,1079
L7,919
L1,765
62,764


In [18]:
levels_to_replace.append(levels[levels.level<15].index)

In [19]:
for x in levels_to_replace:
    salaries_df['level'] = salaries_df['level'].replace(x,"Other")
    
# Check to make sure binning was successful
salaries_df['level'].value_counts()

Other                            5668
L4                               5014
L5                               4871
L3                               3337
L6                               2871
                                 ... 
New grad                           15
Engineer I                         15
Only one level across Netflix      15
Grade 14                           15
PR3                                15
Name: level, Length: 326, dtype: int64

In [62]:
salaries_df['location'].value_counts()

Seattle, WA                      8701
San Francisco, CA                6797
New York, NY                     4562
Redmond, WA                      2649
Mountain View, CA                2275
                                 ... 
San Fernando, LB, Philippines       1
Suwanee, GA                         1
Oxford, MS                          1
Wayne, PA                           1
Hilbert, WI                         1
Name: location, Length: 1050, dtype: int64

In [63]:
#code to replace locations with fewest value _counts
locations_to_replace=[]
locations=salaries_df['location'].value_counts()
locations=pd.DataFrame(locations)
locations.head(20)

,location
"Seattle, WA",8701
"San Francisco, CA",6797
"New York, NY",4562
"Redmond, WA",2649
"Mountain View, CA",2275
"Sunnyvale, CA",2248
"San Jose, CA",2047
"Austin, TX",1527
"Menlo Park, CA",1440
"Cupertino, CA",1431


In [64]:
locations_to_replace.append(locations[locations.location<15].index)

In [65]:
for x in locations_to_replace:
    salaries_df['location'] = salaries_df['location'].replace(x,"Other")
    
# Check to make sure binning was successful
salaries_df['location'].value_counts()

Seattle, WA            8701
San Francisco, CA      6797
New York, NY           4562
Redmond, WA            2649
Other                  2600
                       ... 
Shenzhen, GD, China      15
Pasadena, CA             15
Princeton, NJ            15
Hangzhou, ZJ, China      15
Grand Rapids, MI         15
Name: location, Length: 198, dtype: int64

In [73]:
top_location_list_df=pd.DataFrame(salaries_df['location'].value_counts())

In [74]:
top_location_list_df.reset_index(level=0, inplace=True)
top_location_list_df

,index,location
0,"Seattle, WA",8701
1,"San Francisco, CA",6797
2,"New York, NY",4562
3,"Redmond, WA",2649
4,Other,2600
...,...,...
193,"Shenzhen, GD, China",15
194,"Pasadena, CA",15
195,"Princeton, NJ",15
196,"Hangzhou, ZJ, China",15


In [75]:
top_location_list_df2=top_location_list_df.head(51)

In [76]:
top_location_list_df2=top_location_list_df2.sort_values('index')
top_location_list_df2

,index,location
37,"Amsterdam, NH, Netherlands",237
36,"Arlington, VA",237
23,"Atlanta, GA",532
8,"Austin, TX",1527
11,"Bangalore, KA, India",1199
19,"Bellevue, WA",737
24,"Bengaluru, KA, India",525
30,"Berlin, BE, Germany",342
13,"Boston, MA",1190
47,"Boulder, CO",193


In [77]:

#<option value="Business Analyst">Business Analyst</option>
for i in top_location_list_df2['index']:
    print(f'<option value="{i}">'+ i + '</option>')

<option value="Amsterdam, NH, Netherlands">Amsterdam, NH, Netherlands</option>
<option value="Arlington, VA">Arlington, VA</option>
<option value="Atlanta, GA">Atlanta, GA</option>
<option value="Austin, TX">Austin, TX</option>
<option value="Bangalore, KA, India">Bangalore, KA, India</option>
<option value="Bellevue, WA">Bellevue, WA</option>
<option value="Bengaluru, KA, India">Bengaluru, KA, India</option>
<option value="Berlin, BE, Germany">Berlin, BE, Germany</option>
<option value="Boston, MA">Boston, MA</option>
<option value="Boulder, CO">Boulder, CO</option>
<option value="Cambridge, MA">Cambridge, MA</option>
<option value="Chicago, IL">Chicago, IL</option>
<option value="Cupertino, CA">Cupertino, CA</option>
<option value="Dallas, TX">Dallas, TX</option>
<option value="Denver, CO">Denver, CO</option>
<option value="Dublin, DN, Ireland">Dublin, DN, Ireland</option>
<option value="Hillsboro, OR">Hillsboro, OR</option>
<option value="Houston, TX">Houston, TX</option>
<option va

In [84]:
#elif x=='Data Scientist':
        #    blank_dummies_df['title_Data Scientist'] = 1
for i in top_location_list_df2['index']:
    print(f'elif x=="{i}": blank_dummies_df["title_{i}"] = 1')

elif x=="Amsterdam, NH, Netherlands": blank_dummies_df["title_Amsterdam, NH, Netherlands"] = 1
elif x=="Arlington, VA": blank_dummies_df["title_Arlington, VA"] = 1
elif x=="Atlanta, GA": blank_dummies_df["title_Atlanta, GA"] = 1
elif x=="Austin, TX": blank_dummies_df["title_Austin, TX"] = 1
elif x=="Bangalore, KA, India": blank_dummies_df["title_Bangalore, KA, India"] = 1
elif x=="Bellevue, WA": blank_dummies_df["title_Bellevue, WA"] = 1
elif x=="Bengaluru, KA, India": blank_dummies_df["title_Bengaluru, KA, India"] = 1
elif x=="Berlin, BE, Germany": blank_dummies_df["title_Berlin, BE, Germany"] = 1
elif x=="Boston, MA": blank_dummies_df["title_Boston, MA"] = 1
elif x=="Boulder, CO": blank_dummies_df["title_Boulder, CO"] = 1
elif x=="Cambridge, MA": blank_dummies_df["title_Cambridge, MA"] = 1
elif x=="Chicago, IL": blank_dummies_df["title_Chicago, IL"] = 1
elif x=="Cupertino, CA": blank_dummies_df["title_Cupertino, CA"] = 1
elif x=="Dallas, TX": blank_dummies_df["title_Dallas, TX"] = 1


In [85]:
job_list_df=pd.DataFrame(salaries_df['title'].value_counts())

In [86]:
job_list_df.reset_index(level=0, inplace=True)
job_list_df

,index,title
0,Software Engineer,41231
1,Product Manager,4673
2,Software Engineering Manager,3569
3,Data Scientist,2578
4,Hardware Engineer,2200
5,Product Designer,1516
6,Technical Program Manager,1381
7,Solution Architect,1157
8,Management Consultant,976
9,Business Analyst,885


In [87]:
job_list_df=job_list_df.sort_values('index')
job_list_df

,index,title
9,Business Analyst,885
3,Data Scientist,2578
4,Hardware Engineer,2200
14,Human Resources,364
8,Management Consultant,976
10,Marketing,710
11,Mechanical Engineer,490
5,Product Designer,1516
1,Product Manager,4673
13,Recruiter,451


In [88]:
#<option value="Business Analyst">Business Analyst</option>
for i in job_list_df['index']:
    print(f'<option value="{i}">'+ i + '</option>')

<option value="Business Analyst">Business Analyst</option>
<option value="Data Scientist">Data Scientist</option>
<option value="Hardware Engineer">Hardware Engineer</option>
<option value="Human Resources">Human Resources</option>
<option value="Management Consultant">Management Consultant</option>
<option value="Marketing">Marketing</option>
<option value="Mechanical Engineer">Mechanical Engineer</option>
<option value="Product Designer">Product Designer</option>
<option value="Product Manager">Product Manager</option>
<option value="Recruiter">Recruiter</option>
<option value="Sales">Sales</option>
<option value="Software Engineer">Software Engineer</option>
<option value="Software Engineering Manager">Software Engineering Manager</option>
<option value="Solution Architect">Solution Architect</option>
<option value="Technical Program Manager">Technical Program Manager</option>
